# Scraping NUFORC Data

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import random
from mldrive import mldrive as md

In [4]:
def get_all_source_html():
    states_list = ("AK","AL","AR","AZ","CA","CO","CT","DC","DE","FL","GA","HI","IA","ID",
               "IL","IN","KS","KY","LA","MA","MD","ME","MI","MN","MO","MS","MT","NC",
               "ND","NE","NH","NJ","NM","NV","NY", "OH","OK","OR","PA","RI","SC",
               "SD","TN","TX","UT","VA","VT","WA","WI","WV","WY")
    webpages = []
    for state in states_list:
        source_html = requests.get(f'http://www.nuforc.org/webreports/ndxl{state}.html').text
        webpages.append(source_html)
    return webpages
    
def parse_html(source_html):
    soup = bs(source_html,'html.parser')
    all_text = soup.get_text()
    file_start = all_text.index('Date / Time')
    html = all_text[file_start:]
    return(html)

def make_list(file):
    split = file.split('\n')         # Splits all_text string by whitespace

    for element in split:                # Unfortunately we need to remove the empty strings from the
        if element == '':                # list since they don't ONLY represent blank cells here
            split.remove(element)    
    columns = split[0:7]                 # Grab Columns
    data = split[7:]                     # Grab Data. Data is not usable yet due to the fact that 
                                         # empty cells are not being read.
    return([columns,data])   

def identify_rows(data):
    rows = []
    for value in range(len(data)):       # Searches the data for entries containing ':' after '/'.
        pieces = data[value].split('/')  # Safe to assume only datetime values will follow this
        if ':' in pieces[-1] and len(pieces) == 3:            # format, so we can insert a placeholder string before them
            rows.append('DATETIME')      # to mark where rows begin.
        rows.append(data[value])
    rows.append('DATETIME')
    return rows

def identify_rows_redux(data):
    rows = []
    for value in range(len(data)):       # Searches the data for entries containing ':' after '/'.
        pieces = data[value].split('/')  # Safe to assume only datetime values will follow this
        if ':' in pieces[-1] and len(pieces) == 3:            # format, so we can insert a placeholder string before them
            rows.append('DATETIME')      # to mark where rows begin.
        if val_decider(data[value-1]) == 'Posted' and val_decider(data[value]) == 'Posted':
            rows.append('DATETIME')
        if data[value] != '':
            rows.append(data[value])
    rows.append('DATETIME')
    return rows

def identify_rows_redux_deux(data):
    rows =['DATETIME']
    for value in range(len(data)):
        rows.append(data[value])
        if valid_posted(data[value]) and valid_datetime(data[value+1]):
            rows.append('DATETIME')
        elif valid_posted(data[value]) and valid_posted(data[value+1]):
            rows.append('DATETIME')
    return rows

def valid_datetime(value):
    valid_datetime = False
    pieces = value.split('/')
    if ':' in pieces[-1] and len(pieces) == 3 and len(value) < 15:
        valid_datetime = True
    return valid_datetime


def valid_shape(value): 
    shape_vals = ['Circle', 'Unknown', 'Cigar', 'Other', 'Light', 'Fireball', 'Rectangle',
                    'Teardrop','Oval', 'Disk', 'Sphere', 'Triangle', 'Formation', 'Cross', 'Diamond',
                    'Changing', 'Chevron', 'Egg', 'Flash', 'Cylinder', 'Cone', 'Delta', 'Star']
    valid_shape = False
    if value in shape_vals:
        valid_shape = True
    return valid_shape

def valid_duration(value):
    valid_duration = False
    if value == '':
        return valid_duration
    if value[0] in ['<','>','~']:
        valid_duration = True
        return valid_duration
    try:
        int(value[0])
    except:
        return valid_duration    # as False
    if '/' in value:
        return valid_duration    # as False
    valid_duration = True
    return valid_duration        # set to True if starts with int and has no '/'
    
    
def valid_summary(value):
    valid_summary = False
    if len(value.split(' ')) > 2:
        valid_summary = True
    return valid_summary
        
def valid_posted(value):
    valid_posted = False    
    dates = value.split('/')
    if len(dates) != 3 or len(value) > 10:
        return valid_posted
    for date in dates:
        if len(date) > 2:
            return valid_posted
    valid_posted = True
    return valid_posted
    

def val_decider(value):
    val_type = 'no_type'
    if valid_shape(value):
        val_type = 'Shape'
    elif valid_duration(value):
        val_type = 'Duration'
    elif valid_summary(value):
        val_type = 'Summary'
    elif valid_posted(value):
        val_type = 'Posted'
    return val_type

def identify_columns(columns, rows):
    rows_list = []
    for i in range(len(rows)):
        if rows[i] == 'DATETIME' and i != len(rows)-1:
            curr_row = []                       # Initialize current row as list. List values follow cols.
            curr_row.append(rows[i+1])          # Add the date/time val to curr_row
            curr_row.append(rows[i+2])          # Add the city val to curr_row
            curr_row.append(rows[i+3])          # Add the state val to curr_row
            row_try = 0
            for j in [3,4,5,6]:                 # Add other values to columns if they exist
                if val_decider(rows[i+row_try+4]) == columns[j]:
                    curr_row.append(rows[i+row_try+4])
                    row_try += 1
                else:
                    curr_row.append('')
            rows_list.append(curr_row)
    return rows_list
    # When we make the read me, emphasize the idea of incrementing columns w/o incrementing
    # val_decider to stay pointed at the current value / not skip it.

def identify_columns_redux(columns,rows):
    rows_list = []
    for i in range(len(rows)):
        if rows[i] == 'DATETIME' and i != len(rows)-1:
            count = 0
            curr_row = ['','','','','','','']
            while rows[i+1+count] != 'DATETIME' and count < 7:
                curr_row[count] = rows[i+1+count]
                count+=1
            for value in curr_row:
                if valid_posted(value):
                    end_of_row = curr_row.index(value)
            if end_of_row != 6:
                    curr_row[6] = curr_row[end_of_row]
                    curr_row[end_of_row] = '' 
            rows_list.append(curr_row)
    return rows_list
    

def identify_columns_redux_deux(columns,rows):
    rows_list = []
    for i in range(len(rows)):
        if rows[i] == 'DATETIME' and i != len(rows)-1:
            count = 0
            curr_row = []
            while val_decider(rows[i+count]) != 'Posted' and count < 7:
                curr_row.append(rows[i+1+count])
                count += 1
            if len(curr_row) < 7:
                while len(curr_row) < 7:
                    curr_row.append('')
                for j in [3,4,5,6]:
                    if val_decider(curr_row[j]) != 'no_type':
                        col_type = val_decider(curr_row[j])
                        if col_type != columns[j]:
                            temp = curr_row[columns.index(col_type)]
                            curr_row[columns.index(col_type)] = curr_row[j] 
                            curr_row[j] = temp
            if val_decider(curr_row[-1]) != 'Posted':
                for val in range(len(curr_row)):
                    if val_decider(curr_row[val]) == 'Posted':
                        temp = curr_row[val]
                        curr_row[val] = ''
                        curr_row[-1] = temp
            if val_decider(curr_row[3]) != 'Shape':
                curr_row[3] = ''
            if val_decider(curr_row[4]) != 'Duration':
                curr_row[4] = ''
            if val_decider(curr_row[5]) != 'Summary':
                curr_row[5] = ''
            if len(curr_row[1])>3 and len(curr_row[2]) == 2:
                rows_list.append(curr_row)
    return rows_list

def merge_csvs(df_list):
    none = []
    return none

## Attempt #1

In [5]:
webpages = get_all_source_html()

In [6]:
list_of_data = []

columns = ['Date / Time', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted']

for webpage in webpages:
    html = parse_html(webpage)

    raw_data = make_list(html)
    columns = raw_data[0]

    rows = identify_rows(raw_data[-1])

    clean_data = identify_columns(columns,rows)

    list_of_data.append(clean_data)
    print(f'appended data for {clean_data[1][2]} \n   Sample Row: {clean_data[55]}')
print(len(list_of_data))

appended data for AK 
   Sample Row: ['9/30/18 06:30', 'Palmer', 'AK', 'Light', '10 minutes', '((NUFORC Note:  Witness provides no information in original report.  Date is flawed.  Possible "twinkling" star??  PD))', '10/6/18']
appended data for AL 
   Sample Row: ['4/10/20 00:00', 'Boaz', 'AL', 'Other', '>30 minutes', 'These Orbs had Green lights and White blinking lights.They moved and hovered!They were at approx 30,000 feet high moving from East to W', '5/1/20']
appended data for AR 
   Sample Row: ['9/26/20 21:00', 'Cotter', 'AR', 'Other', '5 minutes', 'Oval/triangle/filled in boomerang shaped spacecraft. Lights flashing in a row, noises too loud for aircraft and for its size.', '11/5/20']
appended data for AZ 
   Sample Row: ['5/5/21 21:17', 'Show Low', 'AZ', '', '', 'MADAR Node 74', '7/31/21']
appended data for CA 
   Sample Row: ['9/26/21 17:47', 'Los Altos', 'CA', '', '', 'MADAR Node 52', '10/19/21']
appended data for CO 
   Sample Row: ['5/21/21 22:00', 'Lakewood', 'CO', 'Tria

appended data for TX 
   Sample Row: ['6/24/21 14:00', 'Abilene', 'TX', 'Teardrop', '10 minutes', 'White egg/teardrop craft seen hovering near Abilene, TX', '8/16/21']
appended data for UT 
   Sample Row: ['8/13/20 01:40', 'Ogden', 'UT', 'Cigar', '30 minutes', 'My husband and I were out on our patio looking at the stars when I noticed this object that was moving from the west the lights were bl', '8/20/20']
appended data for VA 
   Sample Row: ['12/11/20 11:30', 'Summerville', 'VA', 'Diamond', '5 minutes', '((HOAX??))  A bright light appeared , three  colored lights in back. It  zigzagged , dropped to the ground behind hill.', '3/2/21']
appended data for VT 
   Sample Row: ['7/13/19 21:00', 'Arlington', 'VT', 'Light', '10minutes', 'Bright white,star- like,moving and stopping over Southwestern Vermont,changing course easily', '7/25/19']
appended data for WA 
   Sample Row: ['7/21/21 21:24', 'Napavine(?)', 'WA', 'Triangle', '60 seconds', 'White sparkling lights in a triangle stationary i

In [ ]:
# runtime was 2:05

## Attempt #2

In [5]:
webpages = get_all_source_html()

In [6]:
list_of_data = []

columns = ['Date / Time', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted']

for webpage in webpages:
    html = parse_html(webpage)

    raw_data = make_list(html)
    columns = raw_data[0]

    rows = identify_rows_redux_deux(raw_data[-1])

    clean_data = identify_columns_redux_deux(columns,rows)

    list_of_data.append(clean_data)
    print(f'appended data for {clean_data[1][2]} \n   Sample Row: {clean_data[random.randint(0,100)]}')
print(len(list_of_data))

appended data for AK 
   Sample Row: ['2/26/17 22:51', 'Wasilla', 'AK', 'Oval', '2-3 minutes', '3 low and slow moving pulsating orange/reddish orbs.', '3/10/17']
appended data for AL 
   Sample Row: ['10/8/21 22:00', 'Albertville', 'AL', 'Fireball', '5 second', 'The object was round and it looked like an orb or fireball , it had green color to it like a weld spark. I saw it for about 5 seconds w', '10/19/21']
appended data for AR 
   Sample Row: ['9/19/21 05:15', 'Jonesboro', 'AR', '', '', 'MADAR Node 143', '10/19/21']
appended data for AZ 
   Sample Row: ['12/3/21 18:30', 'Sedona', 'AZ', '', '50 or more lights miles long in a line across the night sky. Lasted a couple minutes then disappeared from the head to tail systematica', '', '12/19/21']
appended data for CA 
   Sample Row: ['12/10/21 00:00', 'Tecopa', 'CA', 'Rectangle', '', 'seen dark rectangle moving slowly thru the sky. There was no illumination of light. Kind of like a bird swam or bugs but rectangle.', '12/19/21']
appended 

appended data for VA 
   Sample Row: ['2/25/21 06:15', 'Richmond', 'VA', 'Changing', '', 'Indeterminable Orb In Early Morning Sky, Richmond VA', '3/2/21']
appended data for VT 
   Sample Row: ['1/20/18 01:00', 'Burlington', 'VT', 'Sphere', '30 seconds', 'three red orbs views from the ethan allen tower.  ((anonymous report))', '2/25/20']
appended data for WA 
   Sample Row: ['8/31/21 20:35', 'Spokane', 'WA', 'Fireball', '5 seconds', 'Orange ball of light heading east, fast, silent, then disappeared', '9/7/21']
appended data for WI 
   Sample Row: ['1/20/21 20:00', 'Stevens point', 'WI', 'Circle', '1 minute', 'Driving southbound on I-39 in Stevens Point we witnessed 6 circular large glowing lights that seemed relatively close. Two were to the', '3/2/21']
appended data for WV 
   Sample Row: ['7/19/20 22:00', 'Hico', 'WV', 'Triangle', '1 minute', 'Triangle of lights', '7/23/20']
appended data for WY 
   Sample Row: ['11/11/18 17:58', 'Evanston', 'WY', 'Circle', '18:00', 'When I said, "Hel

In [56]:
huge_list = []
for data in list_of_data:
    for row in data:
        huge_list.append(row)

df = pd.DataFrame(huge_list)
df.columns = columns
df.head(100)   

,Date / Time,City,State,Shape,Duration,Summary,Posted
0,11/20/21 01:00,Willow,AK,Unknown,,I told my son not to shoot it that it was bigg...,12/19/21
1,10/29/21 09:25,Soldotna,AK,Circle,3 minutes,Shiny metallic craft in morning sunlight,11/15/21
2,10/29/21 09:25,Soldotna,AK,Circle,3 minutes,Shiny metallic craft in morning sunlight.,11/15/21
3,9/5/21 23:00,Fairbanks,AK,Sphere,2 minutes,Fairbanks PD sees UFO,10/19/21
4,3/17/21 23:00,Chugiak,AK,Light,2 minutes,Blue lights in sky,3/31/21
...,...,...,...,...,...,...,...
95,12/31/16 23:50,Anchorage,AK,Fireball,20 minutes,"Three bright orange orbs, exhibiting odd fligh...",1/6/17
96,12/31/16 23:45,Wasilla,AK,Oval,3 minutes,Red glowing object over wasilla skies on new y...,1/6/17
97,12/31/16 20:30,Wasilla,AK,Circle,,Lights in the sky.,1/6/17
98,12/23/16 18:10,Delta Junction,AK,Fireball,45 seconds,Seven fireball glowing/pulsating objects were ...,12/30/16


## Export to .csv

In [31]:
key = 'c1eaf3fa7dfd9087a7bbb5f695f9cd'
key_exchange = 'c1b9fa'
email = 'nickcan2296@gmail.com'

b'<html>\r\n<head><title>413 Request Entity Too Large</title></head>\r\n<body>\r\n<center><h1>413 Request Entity Too Large</h1></center>\r\n<hr><center>nginx/1.18.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'


In [57]:
df.to_csv('ufo.csv')

## Sample Questions:

### 1: What City has the most UFO reportings? State?

In [74]:
all_data = df
new_col = all_data['City'] + ', ' + all_data['State']
new_col.columns = ['CityState']
new_col.mode()

0    Las Vegas, NV
1      Phoenix, AZ
dtype: object

In [75]:
all_data['State'].mode()

0    CA
dtype: object